### Manipulação de Dados com DataFrames no PySpark - Transformações

**Objetivo:** Esta seção demonstra como realizar operações comuns com DataFrames no PySpark e apresenta seus equivalentes em SQL.

---

#### Criação a partir de arquivos de um CSV

- DataFrames podem ser criados lendo diretamente de arquivos. O Spark suporta múltiplos formatos de arquivos, como CSV, JSON, Parquet, ORC, Avro, entre outros.


In [0]:
spark

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, DoubleType


# Definindo um esquema para o DataFrame
schema = StructType([
    StructField("Month", StringType(), True),
    StructField("1958", DoubleType(), True),
    StructField("1959", DoubleType(), True),
    StructField("1960", DoubleType(), True)
])

# Carregando o CSV como DataFrame
df = (
    spark
        .read
        .csv(
            "/Volumes/workspace/default/laboratorio-spark/airtravel.csv",
            header=True,
            schema=schema
        )
        .withColumnsRenamed(colsMap={
            '"1958"': '1958', 
            '"1959"': '1959',
            '"1960"': '1960'
        })
)
df.createOrReplaceTempView("airtravel")
display(df)


#### Operações com DataFrames

##### Seleção de Colunas (`select`)

###### Usando `F.col`

No PySpark, `F.col` é utilizado para referenciar colunas programaticamente, especialmente útil quando os nomes das colunas são dinâmicos ou utilizados em expressões.


In [0]:
import pyspark.sql.functions as F

# Selecionar colunas "Month" e "1958" usando F.col
df_select_col = df.select(F.col("Month"), F.col("1958"))
display(df_select_col)


**Equivalente em SQL:**

In [0]:
%sql
SELECT Month, `1958` 
  FROM airtravel


###### Sem `F.col` (Referência Direta)

Colunas podem ser referenciadas diretamente por nome usando strings, o que é simples e direto para seleções básicas.


In [0]:
# Selecionar colunas "Month" e "1958" sem usar F.col, apenas com strings
df_select_strings = df.select("Month", "1958")
display(df_select_strings)


###### Referenciando Colunas Diretamente no DataFrame

Colunas também podem ser acessadas diretamente através do DataFrame, útil em contextos de transformações dentro de expressões mais complexas.


In [0]:
# Selecionar colunas diretamente no DataFrame usando acesso direto
df_direct = df.select(df.Month, df["1958"])
display(df_direct)


---

##### Filtragem de Dados (`filter`)

A função `filter` em PySpark permite aplicar condições para selecionar linhas específicas de um DataFrame.


In [0]:
df.columns

In [0]:
df.printSchema()

In [0]:
# Filtrar linhas onde o número de passageiros em 1958 é maior que 300 usando F.col
df_filter = df.filter(F.col("1958") > 300)
display(df_filter)


**Equivalente em SQL:**

In [0]:
df_filter = spark.sql("""
SELECT *
FROM airtravel
WHERE `1958` > 300""")
display(df_filter)

In [0]:
%sql
SELECT *
FROM airtravel
WHERE `1958` > 300


---

##### Adicionando ou Modificando Colunas (`withColumn`)

O `withColumn` é usado para adicionar novas colunas ou modificar colunas existentes em um DataFrame.


In [0]:
# Adicionar uma nova coluna com incremento de passageiros de 1958 para 1959 usando F.expr
df_with_column = df.withColumn("Incremento", F.expr("`1959` - `1958`"))
display(df_with_column)

df_with_column = df.withColumn("Incremento", F.col("1959") - F.col("1958"))
display(df_with_column)


**Equivalente em SQL:**

In [0]:
%sql
SELECT *, (`1959` - `1958`) AS Incremento
FROM airtravel


---

##### Renomeando Colunas (`withColumnRenamed`)

Renomear colunas em PySpark é feito com `withColumnRenamed`, ideal para limpar e padronizar nomes de colunas.


In [0]:
# Renomear a coluna "1958" para "Ano1958"
df_renamed = df.withColumnRenamed("1958", "Ano1958")
display(df_renamed)


**Equivalente em SQL:**

In [0]:
%sql
SELECT Month, `1958` AS Ano1958, `1959`, `1960`
FROM airtravel


---

##### Removendo Colunas (`drop`)

A função `drop` é usada para remover colunas desnecessárias de um DataFrame, simplificando a estrutura dos dados.


In [0]:
# Remover a coluna "1960" usando drop
df_drop = df.drop("1960")
display(df_drop)


**Equivalente em SQL:**

In [0]:
%sql
SELECT Month, `1958`, `1959`
FROM airtravel


---

##### Removendo Duplicatas (`distinct`)

`distinct` remove linhas duplicadas, assegurando a unicidade dos registros no DataFrame.


In [0]:
# Remover duplicatas no DataFrame
df_distinct = df.distinct()
display(df_distinct)


**Equivalente em SQL:**

In [0]:
%sql
SELECT DISTINCT *
FROM airtravel


---

##### Ordenando Dados (`orderBy`)

`orderBy` organiza os dados em ordem específica, essencial para análises que envolvem classificação ou tendências.


In [0]:
# Ordenar o DataFrame pela coluna "1958" em ordem decrescente usando F.col
df_order = df.orderBy(F.col("1958").desc())
display(df_order)


**Equivalente em SQL:**

In [0]:
%sql
SELECT *
FROM airtravel
ORDER BY `1958` DESC


---

##### Agregações (`groupBy`)

A função `groupBy` agrupa os dados por uma ou mais colunas e permite aplicar funções agregadas como `sum`, `avg`, e `count`.


In [0]:
# Agrupar por 'Month' e somar os valores de "1958" usando F.sum
df_group = (
    df
        .groupBy("Month")
        .agg(
            F.sum("1958").alias("total_1958")
        )
)
display(df_group)


**Equivalente em SQL:**

In [0]:
%sql
SELECT Month, SUM(`1958`) AS total_1958
FROM airtravel
GROUP BY Month


##### Funcões de agrupamento

Documentação de referência: [Aggregate Functions](https://spark.apache.org/docs/latest/sql-ref-functions-builtin.html#aggregate-functions)


In [0]:
df_agg = (
    df
    .agg(
        F.sum("1958").alias("total_1958"), 
        F.avg("1958").alias("avg_1958"), 
        F.mean("1958").alias("mean_1958"), 
        F.median("1958").alias("median_1958"), 
        F.count("1958").alias("count_1958"),
        F.min("1958").alias("min_1958"),
        F.max("1958").alias("max_1958"),
    )
)
display(df_agg)


**Equivalente em SQL:**

In [0]:
%sql
SELECT 
    avg(`1958`) AS avg_1958, 
    mean(`1958`) AS mean_1958, 
    median(`1958`) AS median_1958,
    sum(`1958`) AS total_1958,
    count(`1958`) AS count_1958,
    min(`1958`) AS median_1958,
    max(`1958`) AS median_1958
FROM airtravel


**Resumo das principais funções**
Aqui está uma tabela com o resumo das principais funções de agregação que discutimos:

| Função                       | Descrição                                             | Fórmula/Definição                                               | Exemplo de Dados   | Resultado         |
|------------------------------|-------------------------------------------------------|-----------------------------------------------------------------|--------------------|-------------------|
| **`COUNT`**                  | Conta o número total de elementos                     | $$\text{COUNT} = \text{Número total de elementos}$$             | [1, 2, 3, 4, 5]    | 5                 |
| **`MIN`**                    | Retorna o valor mínimo em um conjunto de dados        | $$\text{MIN} = \text{Valor mínimo em um conjunto}$$             | [3, 5, 1, 8, 2]    | 1                 |
| **`MAX`**                    | Retorna o valor máximo em um conjunto de dados        | $$\text{MAX} = \text{Valor máximo em um conjunto}$$             | [3, 5, 1, 8, 2]    | 8                 |
| **`SUM`**                    | Retorna a soma total dos valores                      | $$\text{SUM} = \sum_{i=1}^{n} x_i$$                             | [3, 5, 1, 8, 2]    | 19                |
| **`AVG`** (Média)            | Retorna a média aritmética dos valores                | $$\text{AVG} = \frac{\sum x_i}{n}$$                             | [1, 2, 3, 4, 5]    | 3                 |
| **`MEAN`**                   | Outra denominação para a média aritmética             | Idêntico ao `AVG`                                               | [1, 2, 3, 4, 5]    | 3                 |
| **`MEDIAN`**                 | Retorna o valor central em um conjunto ordenado       | Valor do meio em um conjunto ordenado                           | [1, 2, 3, 4, 5]    | 3                 |
| **`VAR`** (Variância)        | Mede a dispersão dos dados em relação à média         | $$\text{VAR} = \frac{1}{n}\sum(x_i - \text{mean})^2$$           | [1, 2, 3, 4, 5]    | 2                 |
| **`STDDEV`** (Desvio Padrão) | Mede a dispersão média dos dados                      | $$\text{STDDEV} = \sqrt{\frac{1}{n}\sum(x_i - \text{mean})^2}$$ | [1, 2, 3, 4, 5]    | 1.41              |
| **`FIRST`**                  | Retorna o primeiro valor de uma coluna                | -                                                               | [1, 2, 3, 4, 5]    | 1                 |
| **`LAST`**                   | Retorna o último valor de uma coluna                  | -                                                               | [1, 2, 3, 4, 5]    | 5                 |
| **`COLLECT_LIST`**           | Agrupa valores de uma coluna em uma lista             | -                                                               | [1, 2, 3, 4, 5]    | [1, 2, 3, 4, 5]   |
| **`COLLECT_SET`**            | Agrupa valores distintos de uma coluna em um conjunto | -                                                               | [1, 2, 2, 4, 5]    | {1, 2, 4, 5}      |
| **`PERCENTILE`**             | Retorna o valor de um percentil de uma coluna         | $$\text{PERCENTILE}(p)$$                                        | [1, 2, 3, 4, 5]    |  Depende de $$p$$ |
| **`MODE`**                   | Retorna o valor que ocorre com mais frequência (moda) | -                                                               | [1, 2, 2, 3, 3, 3] | 3                 |

---




##### Junções de DataFrames (`join`)

Junções em PySpark permitem combinar DataFrames com base em chaves comuns, facilitando a integração de dados de diferentes fontes.

In [0]:
df2 = df.withColumnRenamed("1958", "Ano1958").withColumnRenamed("1959", "Ano1959")
df_join = df.join(df2, on="Month", how="inner")
display(df_join)

In [0]:
df2 = df.withColumnRenamed("1958", "Ano1958").withColumnRenamed("1959", "Ano1959")
df_join = df.join(df2, on=["Month"], how="inner") # 2a variacao de como fazer o join
display(df_join)

In [0]:

df2 = df.withColumnRenamed("1958", "Ano1958").withColumnRenamed("1959", "Ano1959")
df_join = (
  df.alias("df")
    .join(
        df2.alias("df2"), 
        F.col("df.Month") == F.col("df2.Month"), 
        how="inner") # 3a variacao de como fazer o join
)
display(df_join)


**Equivalente em SQL:**

In [0]:
%sql
with airtravel2 as (
select 
    month,
    `1958` as ano1958,
    `1959` as ano1959
from airtravel
)
select 
    df.*, 
    df2.ano1958, 
    df2.ano1959
from airtravel as df
    inner join airtravel2 as df2 
        on df2.month = df.month


---

##### Funções de Janela (Window Functions)

Funções de janela são usadas para cálculos que precisam de um contexto mais amplo, como médias móveis ou classificações dentro de grupos.

**Preparando os dados**


In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Criar uma massa de dados
data = [
    ("Marketing", "Alice", 5000),
    ("Marketing", "Bob", 6000),
    ("Marketing", "Charlie", 7000),
    ("Sales", "Dave", 4000),
    ("Sales", "Eve", 4500),
    ("Sales", "Frank", 4800),
    ("IT", "Grace", 5500),
    ("IT", "Heidi", 6000)
]

# Definir o esquema do DataFrame
columns = ["department", "employee", "salary"]

# Criar o DataFrame
employee_df = spark.createDataFrame(data, columns)
employee_df.createOrReplaceTempView("employee")

# Mostrar os dados iniciais
print("Dados iniciais:")
display(employee_df)


**Aplicando row_number dentro de cada departamento**


In [0]:
# Definir a janela de partição por departamento
windowSpec = Window.partitionBy("department").orderBy("salary")

# Aplicar row_number() para numerar as linhas dentro de cada departamento
employee_df_with_row_number = employee_df.withColumn("row_number", F.row_number().over(windowSpec))

# Mostrar o resultado com row_number
print("Aplicando row_number dentro de cada departamento:")
display(employee_df_with_row_number)


**Equivalente em SQL:**


In [0]:
%sql
SELECT
    department,
    employee,
    salary,
    ROW_NUMBER() OVER (PARTITION BY department ORDER BY salary) AS row_number
FROM employee;

In [0]:
# Usar a função lead() para capturar o salário da próxima linha dentro da partição
employee_df_with_next_salary = employee_df_with_row_number.withColumn("next_salary", F.lead("salary").over(windowSpec))

# Mostrar o resultado final com o valor da próxima linha
print("Mostrando o salário da próxima linha dentro de cada departamento:")
display(employee_df_with_next_salary)


**Equivalente em SQL:**


In [0]:
%sql
SELECT 
    department,
    employee,
    salary,
    ROW_NUMBER() OVER (PARTITION BY department ORDER BY salary) AS row_number,
    LEAD(salary) OVER (PARTITION BY department ORDER BY salary) AS next_salary
FROM employee;


---

##### Combinação de DataFrames (`union`)

Documentação de referência: https://spark.apache.org/docs/3.5.2/sql-ref-syntax-qry-select-setops.html

`union` é usado para combinar DataFrames verticalmente, adicionando registros de um DataFrame ao outro.


In [0]:
# União de DataFrame consigo mesmo usando union
df_union = df.union(df)
display(df_union)


**Equivalente em SQL:**

In [0]:
%sql
SELECT * FROM airtravel
UNION ALL
SELECT * FROM airtravel;


---

##### Limitação de Linhas (`limit`)

`limit` restringe o DataFrame a um número específico de linhas, útil para inspeções rápidas de dados.


In [0]:
# Limitar o DataFrame para mostrar apenas as 3 primeiras linhas
df_limit = df.limit(3)
display(df_limit)


**Equivalente em SQL:**

In [0]:
%sql
SELECT *
FROM airtravel
LIMIT 3


---

##### Amostragem Aleatória (`sample`)

A função `sample` cria uma amostra aleatória de linhas do DataFrame, essencial para análises exploratórias.


In [0]:
# Amostra aleatória do DataFrame (50% dos dados) usando sample
df_sample = df.sample(0.5)
display(df_sample)


**Equivalente em SQL:**

In [0]:
%sql
-- Nota: SQL padrão não suporta `SAMPLE` diretamente. Em Spark SQL específico:
SELECT *
FROM airtravel
TABLESAMPLE (50 PERCENT)


**Equivalente em SQL:**

In [0]:
%sql
-- Não há equivalente direto em SQL para reparticionar DataFrames. Esta operação é específica para gerenciamento de desempenho em Spark.


##### Função explode

###### Exemplo 1: Trabalhando com Array

- Preparando os dados


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

# Criar um DataFrame com uma coluna de array
data = [
    ("John", ["Python", "Java", "C++"]),
    ("Alice", ["Scala", "Python"]),
    ("Bob", ["Java", "C#"])
]
columns = ["name", "languages"]

df = spark.createDataFrame(data, columns)

df.createOrReplaceTempView("programadores")

# Mostrar o DataFrame original
print("DataFrame original:")
display(df)


- Usando **`F.explode`** para "explodir" o array


In [0]:
# Explodir a coluna de array "languages"
df_exploded = df.select("name", F.explode(F.col("languages")).alias("language"))

# Mostrar o DataFrame após usar explode
print("DataFrame após explodir o array:")
display(df_exploded)


**Equivalência em SQL **


In [0]:
%sql
SELECT 
  name, 
  EXPLODE(languages) AS language
FROM programadores



###### Exemplo 2: Trabalhando com JSON

- Preparando os dados


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType, ArrayType

# Definindo um esquema para o JSON aninhado
schema = StructType([
    StructField("name", StringType(), True),
    StructField("projects", ArrayType(
        StructType([
            StructField("project_name", StringType(), True),
            StructField("skills", ArrayType(StringType()), True)
        ])
    ), True)
])

# Massa de dados com JSON aninhado
data = [
    ("John", [{"project_name": "AI Project", "skills": ["Python", "TensorFlow"]},
              {"project_name": "Web App", "skills": ["JavaScript", "React"]}]),
    ("Alice", [{"project_name": "Data Analysis", "skills": ["Python", "Pandas"]}])
]

# Criar DataFrame
df = spark.createDataFrame(data, schema)
df.createOrReplaceTempView("funcionarios")

# Mostrar o DataFrame original com JSON aninhado
print("DataFrame original com JSON aninhado:")
display(df)


- Usando **`F.explode`** para "explodir" o array de projetos


In [0]:
# Explodir o array "projects"
df_exploded_projects = (
    df.select(
        "name", 
        F.explode(F.col("projects")).alias("project_info")
    )
)
# Mostrar o DataFrame após explodir o array de projetos
print("DataFrame após explodir o array de projetos:")
display(df_exploded_projects)


**Equivalência em SQL**


In [0]:
%sql
-- Explodir os projetos
SELECT name, EXPLODE(projects) AS project_info
FROM funcionarios;


- Explodir o array de habilidades dentro dos projetos


In [0]:
# Explodir o array "skills" dentro de cada projeto
df_exploded_skills = (
    df_exploded_projects.select(
        "name", 
        "project_info.project_name",
        F.explode(F.col("project_info.skills")).alias("skill")
    )
)
# Mostrar o DataFrame final com todas as habilidades explodidas
print("DataFrame após explodir o array de habilidades:")
display(df_exploded_skills)


**Equivalência em SQL**


In [0]:
%sql
-- Explodir as habilidades dentro de cada projeto
SELECT name, project_info.project_name, EXPLODE(project_info.skills) AS skill
FROM (
    SELECT name, EXPLODE(projects) AS project_info
    FROM funcionarios
);

In [0]:
%sql
with estagio01 as (
  SELECT 
    name, 
    EXPLODE(projects) AS project_info
  FROM funcionarios
)
SELECT 
  name, 
  project_info.project_name, 
  EXPLODE(project_info.skills) AS skill
FROM estagio01;


`[INFO]: FIM DO NOTEBOOK`